Programme de détection de faux billets

In [1]:
import sys
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn import model_selection

Création du modèle de prédiction

In [2]:
#importation du fichier d'entrainement
dataset = pd.read_csv("billet.csv", sep=",", decimal=".", header=0)

#transformation de la variable is_genuine
#True = 0
#False = 1
dataset['is_genuine'] = dataset['is_genuine'].apply(lambda x: 0 if x == True else 1)
dataset["is_genuine"] = dataset["is_genuine"].astype('category')

# on définit x et y
y = dataset["is_genuine"].cat.codes
X = dataset.select_dtypes(np.number)

#subdivision des données en entrainement et test
X_train,X_test,y_train,y_test = model_selection.train_test_split (X,y,test_size = 0.2 ,random_state=0)

#regression logistique
lr = LogisticRegression()

#modélisation sur les données d'apprentissage
modele_sk = lr.fit(X_train, y_train)

Import et contrôle du fichier à tester

In [3]:
from tkinter import * 
from tkinter import filedialog 
from tkinter import messagebox

#sélection du fichier
root = Tk() 
root.filename = filedialog.askopenfilename(initialdir = "C:/Users/OpenClassrooms/jupyter-notebooks/P6_Détectez_des_faux_billets/",
                                           title = "Sélectionnez le fichier",
                                           filetypes = (("Fichiers CSV","*.csv"),("all files","*.*"))) 
root.withdraw()

dataset_new = pd.read_csv(root.filename, sep=",", decimal=".", header=0)

In [4]:
#vérification des variables
liste_X = ["diagonal","height_left","height_right","margin_low","margin_up","length","id"]
if all([item in dataset_new.columns for item in liste_X]):
    print("Les variables du fichier sont correctes. Voici un aperçu des données :\n\n",dataset_new)
else :
    messagebox.showerror(title="Erreur d'importation du fichier", message="Attention, les colonnes du fichier "+root.filename+" ne correspondent pas à la trame attendue !")
    sys.exit()

Les variables du fichier sont correctes. Voici un aperçu des données :

    diagonal  height_left  height_right  margin_low  margin_up  length   id
0    171.76       104.01        103.54        5.21       3.30  111.42  A_1
1    171.87       104.17        104.13        6.00       3.31  112.09  A_2
2    172.00       104.58        104.29        4.99       3.39  111.57  A_3
3    172.49       104.55        104.34        4.44       3.03  113.20  A_4
4    171.65       103.63        103.56        3.77       3.16  113.33  A_5


In [5]:
if((dataset_new.isnull().sum(axis = 0)).sum() > 0):
    messagebox.showerror(title="Valeurs nulles détectées", message="Attention, des valeurs nulles ont été détectées dans le fichier ! Traitement impossible en l'état.")
    sys.exit()
else :
    print("Aucune valeur nulle détectée, le traitement peut continuer.")

Aucune valeur nulle détectée, le traitement peut continuer.


Application de l'algorithme de prédiction

In [6]:
#creation des variables 
X_new = dataset_new[liste_X[:-1]]

#calcul des probabilités d'affectation sur l'échantillon exemple
probas_new = lr.predict_proba(X_new)

#score de presence : Probalilité de chance que le billet soit VRAI (is_genuine = 0)
score_new = probas_new[:,0]

#Ajout de la colonne probabilité
dataset_new["proba_true"] = score_new
#calcul de la variable is_genuine
dataset_new["is_genuine"] = np.where(dataset_new["proba_true"]>=0.5, 'Oui', 'Non')
#traitement du Dataframe pour meilleure compréhension :
dataset_new_fin = dataset_new[["id","proba_true","is_genuine"]].rename(columns={"id":"Identifiant du billet testé",
                                                                                 "proba_true":"Probabilité que le billet soit VRAI (%)",
                                                                                 "is_genuine":"Est authentique ?"})
dataset_new_fin["Probabilité que le billet soit VRAI (%)"] = round((dataset_new_fin["Probabilité que le billet soit VRAI (%)"]*100),2)

Résultats du traitement

Selon la consigne donnée, pour chaque billet, l'algorithme de classification devra donner la probabilité que le billet soit vrai. Si cette probabilité est supérieure ou égale à 0.5 (50%), le billet sera considéré comme vrai. Dans le cas contraire, il sera considéré comme faux.

In [7]:
dataset_new_fin

,Identifiant du billet testé,Probabilité que le billet soit VRAI (%),Est authentique ?
0,A_1,5.10,Non
1,A_2,1.36,Non
2,A_3,3.42,Non
3,A_4,86.73,Oui
4,A_5,99.53,Oui


In [8]:
liste_faux_billets = list(dataset_new_fin.loc[(dataset_new_fin["Est authentique ?"] == "Non"),"Identifiant du billet testé"].values)

In [9]:
print("Les billets dont l'identifiant est dans la liste", liste_faux_billets, "\nont été identifiés commes FAUX selon l'algorithme de prédiction.")

Les billets dont l'identifiant est dans la liste ['A_1', 'A_2', 'A_3'] 
ont été identifiés commes FAUX selon l'algorithme de prédiction.
